In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn import preprocessing
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime, timedelta
from tzlocal import get_localzone 
import shutil
%matplotlib inline

In [2]:
cache_group_id_2_ipmd5=pd.read_csv("machine_pptv_fudan.csv")

In [3]:
cache_group_id_2_ipmd5

,server_id,cache_group_id,server_name,isp_id,area_id,second_level_area,app_service,cache_group_name,type,server_ip_md5,cache_ip_md5
0,253177,105722,xyd112,4,101017,shanxi,live_cold,yd_hb_sx_live_zu1,0,341512d2f669139abb4c6e0363da1968,e44879d95d4cd8e844dc2d3b74c8df70
1,253178,105722,yidong99,4,101017,shanxi,live_cold,yd_hb_sx_live_zu1,0,597bacbbb6773e69e62ab2ffbd46d35f,e44879d95d4cd8e844dc2d3b74c8df70
2,1163500224463849609,109367,PSbjzknd32,10,101010,beijing,live_cold,ck_hb_bj_live_zu3,0,b1236c43ff79c7d6a4e026f817425464,675b3e9cd4f44d94e7b32784c31aa5dd
3,257764,109367,PSbjzklo31,10,101010,beijing,live_cold,ck_hb_bj_live_zu3,0,b964e7ea948da9e6f69ac59c12daa908,675b3e9cd4f44d94e7b32784c31aa5dd
4,257766,109367,PSbjzkda82,10,101010,beijing,live_cold,ck_hb_bj_live_zu3,0,8bfaa8784422eb2ab69f23dc4091689d,675b3e9cd4f44d94e7b32784c31aa5dd
5,257765,109367,PSbjzkuo33,10,101010,beijing,live_cold,ck_hb_bj_live_zu3,0,592035c0cf4fa19470db9840b4aedccb,675b3e9cd4f44d94e7b32784c31aa5dd
6,257763,109367,PSbjzkmr30,10,101010,beijing,live_cold,ck_hb_bj_live_zu3,0,4bf3323056bafa748c993a9d3651ab4f,675b3e9cd4f44d94e7b32784c31aa5dd
7,258483,13280,wzh184,0,101032,guangxi,live_cold,dx_hn_gxwz_live_zu4,0,4a021961f8d04762c3bed4455a68f113,34416cf64158664db36f67eaf482fe33
8,115638,13280,dianxin188,0,101032,guangxi,live_cold,dx_hn_gxwz_live_zu4,0,df4f982269a580c6e2677b56adc7c228,34416cf64158664db36f67eaf482fe33
9,109291,13280,wzh185,0,101032,guangxi,live_cold,dx_hn_gxwz_live_zu4,0,436f42f143279ff2a46a6a11939a5da0,34416cf64158664db36f67eaf482fe33


In [45]:
ip_selected=109771
ip_list=list(cache_group_id_2_ipmd5[cache_group_id_2_ipmd5['cache_group_id']==ip_selected]['server_ip_md5'])
ip_list

['378e706cca6dab8bc10f12b10ca90af0',
 '188f6822360277e8f208ced61f13aad0',
 '466f214346cbcec9a54e5492c3f3ee28',
 '43b98d025f2f10e9e894a85f28f3ce25']

In [35]:
file_dir="./machine/pptv_pstatp_20180107/"  #以28号的数据来说明
for i in ip_list:
    print(pd.read_csv(file_dir+i+".csv").shape[1])

146
146
146
146


In [26]:
date_list=os.listdir("./machine")
date_list_=[]
date_list.sort()
date_list=date_list[::2]
for i,temp in enumerate(date_list):
    date_list[i]=temp[-8:]
    date_list_.append(date_list[i][:4]+'-'+date_list[i][4:6]+'-'+date_list[i][6:])
date_list_
date_list=date_list[-16:]
date_list_=date_list_[-16:]

In [27]:
date_list_

['2018-01-07',
 '2018-01-08',
 '2018-01-09',
 '2018-01-10',
 '2018-01-11',
 '2018-01-12',
 '2018-01-13',
 '2018-01-14',
 '2018-01-15',
 '2018-01-16',
 '2018-01-17',
 '2018-01-18',
 '2018-01-19',
 '2018-01-20',
 '2018-01-21',
 '2018-01-22']

拷贝machine数据

In [10]:
def machine_data_gen(cache_id):
    machine_dir="./"+str(cache_id)+"/machine/"
    ip_list=list(cache_group_id_2_ipmd5[cache_group_id_2_ipmd5['cache_group_id']==cache_id]['server_ip_md5'])
    print(ip_list)
    for i in range(len(date_list_)):
        os.mkdir("./train_data/"+machine_dir+date_list_[i])
        for j in ip_list:
            shutil.copy("./machine/"+"pptv_pstatp_"+date_list[i]+"/"+j+".csv","./train_data/"+machine_dir+date_list_[i])

In [84]:
machine_data_gen(ip_selected)

['73f8ec91aa4b1fc28383d319752c9622', '4de93c9a18508f3c6fccb389f8a44244', 'ff8aa6595b7dc717a0ca2a3a55b73fb4', 'ffb02d32ba0738a8a236bf23899da26a', 'f22f0c511d4752d31d5afb80bacfb31b']


拷贝rate数据

In [11]:
def rate_data_gen(cache_id):
    for i in date_list_:
        dir_="./rate/"+i+"_vc.log"
        hit_rate=pd.read_csv(dir_)
        #hit_rate.dtypes
        #hit_rate[['cache_group_ID']]=hit_rate[['cache_group_ID']].apply(pd.to_numeric)
        hit_rate_dict=dict(list(hit_rate.groupby(['cache_group_ID'])))
        # hit_rate_df=hit_rate_dict[str(cache_id)].sort_values('time')
        hit_rate_df_0=hit_rate_dict[cache_id]
        hit_rate_df_1=hit_rate_dict[str(cache_id)]
        hit_rate_df=pd.concat([hit_rate_df_0,hit_rate_df_1]).sort_values("time")
        
        hit_rate_df=hit_rate_df.loc[hit_rate_df['hit_reach_rate']!='-',:]
        hit_rate_df['hit_reach_cnt']=0
        hit_rate_df[['hit_reach_rate','hit_reach_cnt','hit_cnt']]=hit_rate_df[['hit_reach_rate','hit_reach_cnt','hit_cnt']].apply(pd.to_numeric)
        hit_rate_df['hit_reach_cnt']=hit_rate_df['hit_reach_rate']*hit_rate_df['hit_cnt']/100
        hit_rate_df_1=hit_rate_df[['time','hit_reach_rate','hit_cnt','hit_reach_cnt']]
        hit_rate_df_1.reset_index()
        #hit_rate_df_1.dtypes
        hit_rate_df_1[['hit_cnt','hit_reach_cnt']]=hit_rate_df_1[['hit_cnt','hit_reach_cnt']].apply(pd.to_numeric)
        hit_rate_df_2=hit_rate_df_1.groupby('time',as_index=False).sum()
        hit_rate_df_2['hit_reach_rate']=hit_rate_df_2['hit_reach_cnt']/hit_rate_df_2['hit_cnt']*100
       
        temp=hit_rate_df_2[hit_rate_df_2['hit_cnt']<50]
        temp["hit_reach_rate"]=np.nan
        temp["hit_cnt"]=np.nan
        hit_rate_df_2[hit_rate_df_2['hit_cnt']<50]=temp    #对于请求数小于20的时刻  达标率不做参考

    
        hit_rate_df_2['time']=hit_rate_df_2['time'].astype(dtype='datetime64[ns]')
        hit_rate_df_2.set_index("time", inplace=True)
        
        index=pd.date_range(start=i+' 00:00',end=i+' 23:59' , freq='min')

        full_hit_reach_rate=pd.DataFrame(index=index,dtype=np.float64,columns=['hit_reach_rate'])

        full_hit_reach_rate['hit_reach_rate']=hit_rate_df_2['hit_reach_rate']
        full_hit_reach_rate['hit_cnt']=hit_rate_df_2['hit_cnt']
        full_hit_reach_rate[['hit_cnt']]=full_hit_reach_rate[['hit_cnt']].apply(pd.to_numeric)

        full_hit_reach_rate=full_hit_reach_rate.fillna(method='backfill')
        full_hit_reach_rate=full_hit_reach_rate.reset_index(drop=False)    #恢复索引 时间列不删除
        full_hit_reach_rate.rename(columns={'index':'time'}, inplace = True)

        full_hit_reach_rate.to_csv("./train_data/"+str(cache_id)+"/rate/"+i+".csv")

In [12]:
def machine_rate_data_gen(cache_id):
    #mkdir
    if not os.path.exists("./train_data/"+str(cache_id)):
        os.mkdir("./train_data/"+str(cache_id))
        os.mkdir("./train_data/"+str(cache_id)+"/"+"machine")
        os.mkdir("./train_data/"+str(cache_id)+"/"+"rate")
    #machine_gen
    machine_data_gen(cache_id)
    #rate_gen
    rate_data_gen(cache_id)

In [32]:
machine_rate_data_gen(73843)

['e55927c52e82796951c44dbba72717af', '599b6381d3e54ceafb9c88a49aecdcc6', '73132f885eb2a95f7142d11923ee274f', '8fa5b34112e44738229a807506434d93']


d:\program files\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (2,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
d:\program files\anaconda3\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


在选择分析的cache_group的时候，应该注意cache_group的请求数不能太少，太少了单个请求对达标率的影响就太大，达标率可靠性不高

In [34]:
hit_rate=pd.read_csv("./rate/2018-01-11_vc.log")
#hit_rate.dtypes
#hit_rate[['cache_group_ID']]=hit_rate[['cache_group_ID']].apply(pd.to_numeric)
hit_rate_dict=dict(list(hit_rate.groupby(['cache_group_ID'])))

d:\program files\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
hit_rate_df=hit_rate_dict[str(92317)].sort_values('time')
hit_rate_df=hit_rate_df.loc[hit_rate_df['hit_reach_rate']!='-',:]
hit_rate_df

,view,ip_view,cache_group_ID,cache_view,cache_group_hash,time,hit_reach_rate,hit_cnt,miss_reach_rate,miss_cnt,request_cnt,hit_rate
98490,CN_henan-CN_wangtong,wt_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:00,96.15,26,87.50,7,33,78.78
83690,CN_hubei-CN_wangtong,wt_hubei,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:02,100.00,36,100.00,1,37,97.29
77311,CN_henan-CN_wangtong,wt_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:02,94.11,17,-,0,17,100.00
99373,CN_henan-CN_dianxin,dx_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:02,92.64,5577,93.43,683,6260,89.08
78835,CN_henan-CN_wangtong,wt_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:04,78.94,19,-,0,19,100.00
89373,CN_hubei-CN_wangtong,wt_hubei,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:05,100.00,41,100.00,1,42,97.61
100508,CN_henan-CN_wangtong,wt_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:05,0.00,3,-,0,3,100.00
79726,CN_henan-CN_dianxin,dx_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:05,94.03,5585,93.50,677,6262,89.18
90602,CN_hubei-CN_dianxin,dx_hubei,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:06,93.15,3037,98.02,149,3186,95.32
73051,CN_henan-CN_dianxin,dx_henan,92317,dx_henan,a98660c1bb6f7e8e05e5469ac9611cb7,2018-01-11 00:06,93.53,5739,95.09,736,6475,88.63


machine数据和rate数据都不缺失的cache组的 请求数统计

In [3]:
evaluable_cache_group_list=[              63307,               83346,               53933,
                         105713,               82895,              102742,
                         110040,               79743,              105702,
                          86794,              103721, 1172509055262240814,
                         105730, 1172515855242725417,              108848,
                          86259, 1172508071862267293,              105628,
                          11292, 1168008088666573720, 1172511687907009984,
                         109771,   19600969245461398,               83077,
                          73843,              104780,               20814,
                         105710,               82897,               88170,
                          94161,              104798,              109940,
                          87569,               99529,               77484,
                          53917,              108850, 1163503507880657011,
                          86856,               83326,               72010,
                         103722,               53914,               92317,
                          90109,               77680,              105701]

In [4]:
evaluable_cache_group_list.reverse()

In [5]:
dir_="./rate/"+"2018-01-07"+"_vc.log"
hit_rate=pd.read_csv(dir_)
#hit_rate.dtypes
#hit_rate[['cache_group_ID']]=hit_rate[['cache_group_ID']].apply(pd.to_numeric)
hit_rate_dict=dict(list(hit_rate.groupby(['cache_group_ID'])))

d:\program files\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
con_list=[]
for group in evaluable_cache_group_list:
    #hit_rate_df=hit_rate_dict[str(group)].sort_values('time')
    
    hit_rate_df_0=hit_rate_dict[group]
    hit_rate_df_1=hit_rate_dict[str(group)]
    hit_rate_df=pd.concat([hit_rate_df_0,hit_rate_df_1]).sort_values("time")
    
    hit_rate_df=hit_rate_df.loc[hit_rate_df['hit_reach_rate']!='-',:]
    hit_rate_df['hit_cnt']=hit_rate_df['hit_cnt'].apply(pd.to_numeric)
    hit_rate_df_1=hit_rate_df.groupby('time',as_index=False).sum()
    #print(group)
    a=hit_rate_df_1["hit_cnt"].describe(percentiles=[0,0.05,0.1,0.2,0.5,0.8,1])
    #a.rename(columns={"hit_cnt": group})
    con_list.append(a)

cache_hit_cnt=pd.concat(con_list,axis=1)
cache_hit_cnt.columns = evaluable_cache_group_list

In [7]:
cache_hit_cnt.to_csv("cache_hit_cnt_view.csv")

In [126]:
hit_rate_df=hit_rate_dict[str(103721)].sort_values('time')
hit_rate_df=hit_rate_df.loc[hit_rate_df['hit_reach_rate']!='-',:]
hit_rate_df_1[['hit_cnt','hit_reach_cnt']]=hit_rate_df_1[['hit_cnt','hit_reach_cnt']].apply(pd.to_numeric)

In [128]:
hit_rate_df_2=hit_rate_df_1.groupby('time',as_index=False).sum()